# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 6 2022 9:25AM,241177,10,SOTAH0000723,"Nextsource Biotechnology, LLC",Released
1,Jul 6 2022 9:25AM,241177,10,SONSB0001852,"Nextsource Biotechnology, LLC",Released
2,Jul 6 2022 9:25AM,241177,10,SONSB0001853,"Nextsource Biotechnology, LLC",Released
3,Jul 6 2022 9:24AM,241176,12,HH-35693,Hush Hush,Released
4,Jul 6 2022 9:24AM,241175,12,HH-35694,Hush Hush,Released
5,Jul 6 2022 9:24AM,241175,12,HH-35695,Hush Hush,Released
6,Jul 6 2022 9:24AM,241175,12,HH-35696,Hush Hush,Released
7,Jul 6 2022 9:24AM,241175,12,HH-35508,Hush Hush,Released
8,Jul 6 2022 9:24AM,241175,12,HH-35685,Hush Hush,Released
9,Jul 6 2022 9:24AM,241175,12,HH-35688,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
75,241171,Released,1
76,241174,Released,1
77,241175,Released,10
78,241176,Released,1
79,241177,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241171,NaN,NaN,1.0
241174,NaN,NaN,1.0
241175,NaN,NaN,10.0
241176,NaN,NaN,1.0
241177,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241049,0.0,0.0,55.0
241050,0.0,0.0,55.0
241055,0.0,0.0,55.0
241056,0.0,0.0,49.0
241058,0.0,0.0,51.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241049,0,0,55
241050,0,0,55
241055,0,0,55
241056,0,0,49
241058,0,0,51


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241049,0,0,55
1,241050,0,0,55
2,241055,0,0,55
3,241056,0,0,49
4,241058,0,0,51


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241049,,,55
1,241050,,,55
2,241055,,,55
3,241056,,,49
4,241058,,,51


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 6 2022 9:25AM,241177,10,"Nextsource Biotechnology, LLC"
3,Jul 6 2022 9:24AM,241176,12,Hush Hush
4,Jul 6 2022 9:24AM,241175,12,Hush Hush
14,Jul 6 2022 9:18AM,241174,10,"Senseonics, Incorporated"
15,Jul 6 2022 9:18AM,241169,20,Alumier Labs Inc.
47,Jul 6 2022 9:13AM,241171,10,Prodigy Innovation LLC
48,Jul 6 2022 9:12AM,241170,10,"Citieffe, Inc."
49,Jul 6 2022 9:08AM,241164,10,Bio-PRF
57,Jul 6 2022 9:03AM,241163,10,ISDIN Corporation
115,Jul 6 2022 8:48AM,241165,18,Eye Pharma Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 6 2022 9:25AM,241177,10,"Nextsource Biotechnology, LLC",,,3
1,Jul 6 2022 9:24AM,241176,12,Hush Hush,,,1
2,Jul 6 2022 9:24AM,241175,12,Hush Hush,,,10
3,Jul 6 2022 9:18AM,241174,10,"Senseonics, Incorporated",,,1
4,Jul 6 2022 9:18AM,241169,20,Alumier Labs Inc.,,,32
5,Jul 6 2022 9:13AM,241171,10,Prodigy Innovation LLC,,,1
6,Jul 6 2022 9:12AM,241170,10,"Citieffe, Inc.",,,1
7,Jul 6 2022 9:08AM,241164,10,Bio-PRF,,,8
8,Jul 6 2022 9:03AM,241163,10,ISDIN Corporation,,,58
9,Jul 6 2022 8:48AM,241165,18,Eye Pharma Inc,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 9:25AM,241177,10,"Nextsource Biotechnology, LLC",3,,
1,Jul 6 2022 9:24AM,241176,12,Hush Hush,1,,
2,Jul 6 2022 9:24AM,241175,12,Hush Hush,10,,
3,Jul 6 2022 9:18AM,241174,10,"Senseonics, Incorporated",1,,
4,Jul 6 2022 9:18AM,241169,20,Alumier Labs Inc.,32,,
5,Jul 6 2022 9:13AM,241171,10,Prodigy Innovation LLC,1,,
6,Jul 6 2022 9:12AM,241170,10,"Citieffe, Inc.",1,,
7,Jul 6 2022 9:08AM,241164,10,Bio-PRF,8,,
8,Jul 6 2022 9:03AM,241163,10,ISDIN Corporation,58,,
9,Jul 6 2022 8:48AM,241165,18,Eye Pharma Inc,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 9:25AM,241177,10,"Nextsource Biotechnology, LLC",3,,
1,Jul 6 2022 9:24AM,241176,12,Hush Hush,1,,
2,Jul 6 2022 9:24AM,241175,12,Hush Hush,10,,
3,Jul 6 2022 9:18AM,241174,10,"Senseonics, Incorporated",1,,
4,Jul 6 2022 9:18AM,241169,20,Alumier Labs Inc.,32,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 9:25AM,241177,10,"Nextsource Biotechnology, LLC",3.0,NaN,NaN
1,Jul 6 2022 9:24AM,241176,12,Hush Hush,1.0,NaN,NaN
2,Jul 6 2022 9:24AM,241175,12,Hush Hush,10.0,NaN,NaN
3,Jul 6 2022 9:18AM,241174,10,"Senseonics, Incorporated",1.0,NaN,NaN
4,Jul 6 2022 9:18AM,241169,20,Alumier Labs Inc.,32.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 9:25AM,241177,10,"Nextsource Biotechnology, LLC",3.0,0.0,0.0
1,Jul 6 2022 9:24AM,241176,12,Hush Hush,1.0,0.0,0.0
2,Jul 6 2022 9:24AM,241175,12,Hush Hush,10.0,0.0,0.0
3,Jul 6 2022 9:18AM,241174,10,"Senseonics, Incorporated",1.0,0.0,0.0
4,Jul 6 2022 9:18AM,241169,20,Alumier Labs Inc.,32.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,10367979,1005.0,45.0,3.0
12,1446758,29.0,0.0,0.0
16,1929009,13.0,4.0,0.0
18,482320,2.0,0.0,0.0
19,723389,80.0,22.0,2.0
20,1446622,129.0,15.0,28.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,10367979,1005.0,45.0,3.0
1,12,1446758,29.0,0.0,0.0
2,16,1929009,13.0,4.0,0.0
3,18,482320,2.0,0.0,0.0
4,19,723389,80.0,22.0,2.0
5,20,1446622,129.0,15.0,28.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,1005.0,45.0,3.0
1,12,29.0,0.0,0.0
2,16,13.0,4.0,0.0
3,18,2.0,0.0,0.0
4,19,80.0,22.0,2.0
5,20,129.0,15.0,28.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1005.0
1,12,Released,29.0
2,16,Released,13.0
3,18,Released,2.0
4,19,Released,80.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20
Status,,,,,,
Completed,3.0,0.0,0.0,0.0,2.0,28.0
Executing,45.0,0.0,4.0,0.0,22.0,15.0
Released,1005.0,29.0,13.0,2.0,80.0,129.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20
0,Completed,3.0,0.0,0.0,0.0,2.0,28.0
1,Executing,45.0,0.0,4.0,0.0,22.0,15.0
2,Released,1005.0,29.0,13.0,2.0,80.0,129.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20
0,Completed,3.0,0.0,0.0,0.0,2.0,28.0
1,Executing,45.0,0.0,4.0,0.0,22.0,15.0
2,Released,1005.0,29.0,13.0,2.0,80.0,129.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()